In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


plt.rc('font', family='Malgun Gothic') # For Windows
print(plt.rcParams['font.family'])

%matplotlib inline

['Malgun Gothic']


In [2]:
# 지수표현 제거
pd.options.display.float_format = '{:.5f}'.format

In [3]:
log_data = pd.read_csv('data/log_data.csv')

# 1. 로그 데이터 전처리

## 1.1. log data 전처리
- mp_os, mp_app_version 열 제거
- timestamp에서 시간만 추출

In [4]:
log_data.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25


In [5]:
# 중복된 데이터 제거
log_data = log_data.drop_duplicates().reset_index(drop=True)

# mp_os, mp_app_version 제거
log_data = log_data.drop(['mp_os', 'mp_app_version'], axis=1)

In [6]:
# user_id, 시간대별로 정렬해준다
log_data_sort = log_data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)


In [7]:
from tqdm import tqdm
from datetime import datetime

In [8]:
# 각 user_id별로 전체 로그 데이터를 검사하기 위해 user_id 찾아서 넣어준다
log_data_users = log_data_sort['user_id'].unique()

## 1.2. 군집용 log데이터 생성
- 1. 각 이벤트들의 간격이 15분(900초) 이상이라면 다른 행동이라고 가정
- 2. 따라서 15분 이내에 발생한 user의 이벤트(event)와 시간(timestamp)들을 하나의 user의 데이터로 합침

In [9]:
# 군집용 데이터프레임 틀을 만들어준다
user_activity_df = pd.DataFrame(columns=['user_id', 'date', 'events', 'timestamp'])
user_activity_df

,user_id,date,events,timestamp


In [10]:
file_index = 1
iteration = 0
checkpoint_start = 0
checkpoint_end = 0
for user_id in tqdm(log_data_users):
    
    # log_data_sort의 row를 다 돌때까지
    while checkpoint_end < len(log_data_sort):
        # 이미 정렬해놓은 데이터프레임에서, 같은 user끼리의 임시 데이터프레임을 만들기 위함
        if log_data_sort.loc[checkpoint_end, :]['user_id'] == user_id:
            checkpoint_end += 1
        else:
            break
            
    # 임시 데이터프레임
    tmp = log_data_sort.iloc[checkpoint_start:checkpoint_end+1, :].reset_index(drop=True)
    
    pivot = tmp.iloc[0, :]
    event_save = pivot['event']
    time_save = pivot['timestamp']
    
    
    # 처음 df 초기화
    tmp_df = pd.DataFrame( {'user_id': [user_id], 'date': [pivot['date_cd']], 'events': ['']} )
    # 시간 check
    for row_index in range(1, len(tmp)):
        row = tmp.iloc[row_index, :]
        dt1 = datetime.strptime(pivot['timestamp'], '%Y-%m-%d %H:%M:%S')
        dt2 = datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
        
        # 두 timestamp의 시간차가 15분이 되질 않는다면
        if (dt2 - dt1).seconds < 900:
            event_save += ("," + row['event'])
            time_save += ("," + row['timestamp'])
            
        else:
            # 이제까지 한 거 저장
            tmp_df['events'] = event_save
            tmp_df['timestamp'] = time_save
            # 임시 데이터프레임과 기존 데이터프레임을 concat
            user_activity_df = pd.concat([user_activity_df, tmp_df], ignore_index = True)
            # 임시 데이터프레임 초기화
            tmp_df = pd.DataFrame( {'user_id': [user_id], 'date': [row['date_cd']], 'events': ['']} )
            event_save = row['event']
            time_save = row['timestamp']
        pivot = row
        
    # 다음 user_id를 선택하기 위해 checkpoint start 갱신
    checkpoint_start = checkpoint_end
    iteration += 1
    
    # 데이터프레임을 계속해서 concat해서 커지게 되면, concat할 때마다 overhead가 발생해서 processing 속도 느려짐
    # 따라서 3000개의 user_id데이터를 처리할 때마다 임시파일로 저장해둔다
    if iteration > 0 and iteration % 3000 == 0:
        user_activity_df.to_csv(f'data/new_log/log_events_{file_index}.csv', index=False)
        user_activity_df = pd.DataFrame(columns=['user_id', 'date', 'events', 'timestamp'])
        print(f'log_events_{file_index}.csv completed!')
        file_index += 1
        
user_activity_df.to_csv(f'data/new_log/log_events_{file_index}.csv', index=False)



  1%|          | 3013/584636 [01:05<3:24:10, 47.48it/s]

log_events_1.csv completed!


  1%|          | 6012/584636 [02:08<2:52:28, 55.92it/s]

log_events_2.csv completed!


  2%|▏         | 9008/584636 [03:20<5:08:14, 31.12it/s] 

...

 99%|█████████▉| 579005/584636 [4:16:11<04:38, 20.24it/s]

log_events_193.csv completed!


100%|█████████▉| 582012/584636 [4:18:10<01:27, 30.15it/s]

log_events_194.csv completed!


100%|██████████| 584636/584636 [4:19:58<00:00, 37.48it/s]


In [12]:
# 임시파일을 다시 하나로 합쳐주는 작업을 한다
log_event_df = pd.DataFrame(columns=['user_id', 'date', 'events', 'timestamp'])
for i in range(1, 195+1):
    tmp_df = pd.read_csv(f"data/new_log/log_events_{i}.csv")
    log_event_df = pd.concat([log_event_df, tmp_df]).reset_index(drop=True)

# 중간 저장
log_event_df.to_csv('data/new_log_events.csv', index=False)
    

## 1.3 군집을 위한 추가 전처리작업
- OpenApp 제거(너무 당연한 로그라서(어플을 키는 것 자체가 OpenApp을 수반하기 때문) 일종의 불용어 처리
- 각 row의 timestamp에서 event 지속시간 추출

In [13]:
# 중간 저장 파일 로드
log_event_df = pd.read_csv('data/new_log_events.csv')
log_event_df

,user_id,date,events,timestamp
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35"
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0..."
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0..."
4,11,2022-03-24,"OpenApp,GetCreditInfo,UseLoanManage,GetCreditI...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0..."
...,...,...,...,...
3608000,879693,2022-06-29,OpenApp,2022-06-29 17:20:45
3608001,879694,2022-03-31,"StartLoanApply,ViewLoanApplyIntro,CompleteIDCe...","2022-03-31 20:07:23,2022-03-31 20:07:23,2022-0..."
3608002,879695,2022-05-27,"StartLoanApply,ViewLoanApplyIntro,CompleteIDCe...","2022-05-27 12:48:32,2022-05-27 12:48:32,2022-0..."
3608003,879696,2022-03-14,"Login,UseLoanManage,GetCreditInfo,GetCreditInf...","2022-03-14 05:35:34,2022-03-14 05:35:41,2022-0..."


In [14]:
# 시간 길이 측정하기
from datetime import datetime

def check_time_length(row):
    time_list = row.split(',')
    
    # event가 1개인 경우일 때는 0초로 처리
    if len(time_list) < 2:
        return 0
    else:
        start_time = time_list[0]
        end_time   = time_list[-1]
        sec = ( datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S') - datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') ).seconds
        return sec

In [15]:
# 각 행마다 이벤트 지속시간 측정(초단위)
log_event_df['time'] = log_event_df['timestamp'].apply(lambda x: check_time_length(x))
log_event_df.head()

,user_id,date,events,timestamp,time
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354
4,11,2022-03-24,"OpenApp,GetCreditInfo,UseLoanManage,GetCreditI...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692


In [16]:
# openapp을 events에서 제거
log_event_df['events'] = log_event_df['events'].apply(lambda x: x.replace('OpenApp,', ''))
log_event_df['events'] = log_event_df['events'].apply(lambda x: x.replace(',OpenApp', ''))
log_event_df['events'] = log_event_df['events'].apply(lambda x: x.replace('OpenApp', ''))

log_event_df.head()

,user_id,date,events,timestamp,time
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354
4,11,2022-03-24,"GetCreditInfo,UseLoanManage,GetCreditInfo,UseP...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692


In [17]:
only_openapp = log_event_df[log_event_df['events'].apply(lambda x: len(x)) == 0].reset_index(drop=True)
# openapp time은 0으로 통일
only_openapp['time'] = 0
only_openapp.head()

,user_id,date,events,timestamp,time
0,20,2022-03-15,,2022-03-15 16:21:32,0
1,20,2022-03-15,,2022-03-15 17:47:51,0
2,20,2022-03-17,,2022-03-17 17:14:47,0
3,20,2022-04-21,,2022-04-21 20:20:49,0
4,20,2022-05-05,,2022-05-05 15:25:31,0


In [18]:
# 기존 데이터(log_event_df) OpenApp만 있는 경우의 수 제거!
new_log_event_df = log_event_df.drop(log_event_df[log_event_df['events'].apply(lambda x: len(x)) == 0].index).reset_index(drop=True)
new_log_event_df

,user_id,date,events,timestamp,time
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354
4,11,2022-03-24,"GetCreditInfo,UseLoanManage,GetCreditInfo,UseP...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692
...,...,...,...,...,...
3055454,879693,2022-06-27,"StartLoanApply,ViewLoanApplyIntro,CompleteIDCe...","2022-06-27 10:13:48,2022-06-27 10:13:54,2022-0...",24
3055455,879694,2022-03-31,"StartLoanApply,ViewLoanApplyIntro,CompleteIDCe...","2022-03-31 20:07:23,2022-03-31 20:07:23,2022-0...",655
3055456,879695,2022-05-27,"StartLoanApply,ViewLoanApplyIntro,CompleteIDCe...","2022-05-27 12:48:32,2022-05-27 12:48:32,2022-0...",171
3055457,879696,2022-03-14,"Login,UseLoanManage,GetCreditInfo,GetCreditInf...","2022-03-14 05:35:34,2022-03-14 05:35:41,2022-0...",400


## 1.4. Glove 활용
- 카운트 기반과 예측 기반 모두 사용하는 방법론
- events의 문자열은 10여개의 단어로만 구성된 문자열
- events를 일일히 전처리하기 보다는, 임베딩 벡터를 활용
- 2단계로 임베딩함 : 10여개의 단어 임베딩 -> 단어 임베딩을 바탕으로 문장(events)를 통채로 임베딩

In [19]:
# # https://velog.io/@ann9902/Glove-%EC%84%A4%EC%B9%98-%EC%97%90%EB%9F%AC-%ED%95%B4%EA%B2%B0%EB%B2%95
# ! pip install glove_python_binary

In [20]:
from glove import Corpus, Glove

In [21]:
# 단어 단위 split
result = [events.split(',') for events in new_log_event_df.loc[:, 'events']]

corpus = Corpus()
corpus.fit(result, window=5)

In [22]:
# 훈련
## no_components는 임베딩 벡터 dimension
glove = Glove(no_components=32, learning_rate=0.01)     # 0.05
%time glove.fit(corpus.matrix, epochs=100, no_threads=4, verbose=False)    # Wall time: 8min 32s
glove.add_dictionary(corpus.dictionary)




Wall time: 24 ms


In [23]:
# # model save
# glove.save('glove_32_epoch100.model')

In [24]:
#model load
glove_model = Glove.load('glove_32_epoch100.model')

In [25]:
# word dict 생성

In [26]:
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

[Success !] Lengh of word dict... :  10


In [27]:
# 훈련시킨 glove model의 단어 및 대응되는 임베딩 벡터 표시
word_dict

{'GetCreditInfo': array([-0.51256538, -0.51179434, -0.49176959,  0.51354897, -0.51007658,
         0.53767853,  0.5127457 ,  0.50309013,  0.50984219,  0.49872453,
         0.50695612,  0.50368135, -0.52142305, -0.51020081,  0.52667964,
         0.52412489,  0.50351871,  0.48969901,  0.52049258, -0.50656359,
        -0.50241932, -0.53583172, -0.52180443, -0.50781765,  0.51770425,
         0.50871608, -0.49971704, -0.50146263,  0.48732218, -0.51015109,
         0.52351302, -0.5303776 ]),
 'UseLoanManage': array([-0.49470323, -0.50163492, -0.491423  ,  0.50558585, -0.51842026,
         0.50611289,  0.51407371,  0.48189495,  0.48874736,  0.5070971 ,
         0.49648585,  0.50529098, -0.51998235, -0.50707956,  0.51682825,
         0.50781484,  0.51227228,  0.47680176,  0.50457247, -0.51463778,
        -0.50713232, -0.51849126, -0.51313478, -0.49172867,  0.52452718,
         0.48606383, -0.49986497, -0.49087542,  0.47712826, -0.49803301,
         0.51246036, -0.52210129]),
 'Login': array([-

In [28]:
def sent2vec_glove(tokens, embedding_dim=32):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

In [29]:
sentence_glove = sent2vec_glove(result)
sentence_glove[0]

array([-0.51256538, -0.51179434, -0.49176959,  0.51354897, -0.51007658,
        0.53767853,  0.5127457 ,  0.50309013,  0.50984219,  0.49872453,
        0.50695612,  0.50368135, -0.52142305, -0.51020081,  0.52667964,
        0.52412489,  0.50351871,  0.48969901,  0.52049258, -0.50656359,
       -0.50241932, -0.53583172, -0.52180443, -0.50781765,  0.51770425,
        0.50871608, -0.49971704, -0.50146263,  0.48732218, -0.51015109,
        0.52351302, -0.5303776 ])

# 2. k-means 군집화
- 우선 events에서 32차원으로 임베딩벡터를 추출한 값을 토대로 k-means를 활용해 log데이터를 우선 군집분석
- log데이터에서 유저의 행동양식을 분류하기 위함

In [30]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

## 2.1 전체 데이터 군집화 (k=5)

In [31]:
# clustering
# 전체 데이터

k = 5
kmeans = KMeans(n_clusters=k, random_state=2022, init = 'k-means++')
y_pred = kmeans.fit_predict(sentence_glove)

In [32]:
new_log_event_df['cluster'] = y_pred
new_log_event_df.head()

,user_id,date,events,timestamp,time,cluster
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7,1
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1,1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0,1
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354,1
4,11,2022-03-24,"GetCreditInfo,UseLoanManage,GetCreditInfo,UseP...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692,1


In [33]:
# 0번 클러스터 : 기타
print(len(new_log_event_df[new_log_event_df['cluster'] == 0]))
new_log_event_df[new_log_event_df['cluster'] == 0]['events'].value_counts()[:30]

731618


Login,GetCreditInfo                                                                                                       154574
Login,GetCreditInfo,GetCreditInfo                                                                                          21562
GetCreditInfo,Login                                                                                                        14160
Login,UseLoanManage,GetCreditInfo,StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply                   13336
Login,GetCreditInfo,StartLoanApply,ViewLoanApplyIntro                                                                      11411
Login,GetCreditInfo,UseLoanManage,StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply                    7748
Login,UseLoanManage,GetCreditInfo,ViewLoanApplyIntro                                                                        5745
Login,UseLoanManage,GetCreditInfo,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply        

In [34]:
# 1번 클러스터 : 부가서비스(getcredit , useloan)
print(len(new_log_event_df[new_log_event_df['cluster'] == 1]))
new_log_event_df[new_log_event_df['cluster'] == 1]['events'].value_counts()[:30]

902713


GetCreditInfo                                                    179540
Login,UseLoanManage,GetCreditInfo                                122819
Login,GetCreditInfo,UseLoanManage                                 63477
GetCreditInfo,GetCreditInfo                                       56647
Login,UseLoanManage                                               53499
GetCreditInfo,UseLoanManage                                       43560
UseLoanManage,GetCreditInfo                                       36187
Login,GetCreditInfo,UseLoanManage,GetCreditInfo                   30408
Login,UseLoanManage,GetCreditInfo,GetCreditInfo                   25556
UseLoanManage,Login,GetCreditInfo                                 16714
GetCreditInfo,GetCreditInfo,GetCreditInfo                         16264
GetCreditInfo,Login,UseLoanManage                                 12503
Login,GetCreditInfo,GetCreditInfo,UseLoanManage                    9581
UseLoanManage                                                   

In [35]:
# 2번 클러스터 : start - endloan
print(len(new_log_event_df[new_log_event_df['cluster'] == 2]))
new_log_event_df[new_log_event_df['cluster'] == 2]['events'].value_counts()[:30]

1229283


Login,StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply                                      217816
StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply                                            115784
Login,StartLoanApply,EndLoanApply                                                                                  99071
StartLoanApply,EndLoanApply                                                                                        45835
Login,ViewLoanApplyIntro                                                                                           39653
Login,GetCreditInfo,StartLoanApply,ViewLoanApplyIntro,CompleteIDCertification,EndLoanApply                         39375
EndLoanApply                                                                                                       36063
Login,StartLoanApply,ViewLoanApplyIntro                                                                            35850
Login,StartLoanApply,ViewLoanApp

In [36]:
# 3번 클러스터 : 신규회원가입
print(len(new_log_event_df[new_log_event_df['cluster'] == 3]))
new_log_event_df[new_log_event_df['cluster'] == 3]['events'].value_counts()[:30]

18755


CompleteIDCertification,SignUp                                  7951
GetCreditInfo,CompleteIDCertification,SignUp                    1191
SignUp                                                          1035
SignUp,GetCreditInfo                                             501
SignUp,UseLoanManage                                             445
GetCreditInfo,UseLoanManage,UsePrepayCalc                        426
GetCreditInfo,UseLoanManage,UseDSRCalc                           412
Login,SignUp                                                     252
SignUp,UseLoanManage,GetCreditInfo                               233
UseLoanManage,GetCreditInfo,CompleteIDCertification,SignUp       140
SignUp,ViewLoanApplyIntro                                        140
SignUp,GetCreditInfo,UseLoanManage                               138
Login,SignUp,UseLoanManage                                       132
GetCreditInfo,UseLoanManage,CompleteIDCertification,SignUp       124
SignUp,GetCreditInfo,UseLoanManage

In [37]:
# 4번클러스터 : 주로 login
print(len(new_log_event_df[new_log_event_df['cluster'] == 4]))
new_log_event_df[new_log_event_df['cluster'] == 4]['events'].value_counts()[:30]

173090


Login                                                                159373
Login,Login                                                            6286
Login,Login,Login                                                      1221
Login,Login,StartLoanApply,ViewLoanApplyIntro                           980
Login,StartLoanApply,ViewLoanApplyIntro,Login                           624
Login,Login,Login,Login                                                 564
Login,ViewLoanApplyIntro,Login                                          369
Login,Login,Login,Login,Login                                           348
Login,GetCreditInfo,Login,Login                                         304
Login,Login,ViewLoanApplyIntro                                          261
Login,Login,Login,Login,Login,Login                                     246
Login,Login,Login,Login,Login,Login,Login                               144
Login,Login,Login,GetCreditInfo                                         128
Login,Login,

In [38]:
# 어플만 킨 경우(openapp) cluster = 5로 해주자
only_openapp['cluster'] = 5
only_openapp.head()

,user_id,date,events,timestamp,time,cluster
0,20,2022-03-15,,2022-03-15 16:21:32,0,5
1,20,2022-03-15,,2022-03-15 17:47:51,0,5
2,20,2022-03-17,,2022-03-17 17:14:47,0,5
3,20,2022-04-21,,2022-04-21 20:20:49,0,5
4,20,2022-05-05,,2022-05-05 15:25:31,0,5


In [39]:
# 기존 df와 openapp만 있는 df을 합쳐준다
new_log_event_df_merge = pd.concat([new_log_event_df, only_openapp]).reset_index(drop=True)
new_log_event_df_merge.head()

,user_id,date,events,timestamp,time,cluster
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7,1
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1,1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0,1
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354,1
4,11,2022-03-24,"GetCreditInfo,UseLoanManage,GetCreditInfo,UseP...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692,1


In [40]:
# user_id, date 기준으로 정렬해준다
new_log_event_df_merge = new_log_event_df_merge.sort_values(by=['user_id', 'date']).reset_index(drop=True)
new_log_event_df_merge.head()

,user_id,date,events,timestamp,time,cluster
0,1,2022-05-03,"GetCreditInfo,GetCreditInfo","2022-05-03 14:52:28,2022-05-03 14:52:35",7,1
1,1,2022-06-16,"UseLoanManage,Login,GetCreditInfo","2022-06-16 23:58:41,2022-06-16 23:58:41,2022-0...",1,1
2,7,2022-05-22,GetCreditInfo,2022-05-22 16:39:49,0,1
3,9,2022-05-21,"GetCreditInfo,GetCreditInfo,GetCreditInfo","2022-05-21 23:37:58,2022-05-21 23:43:33,2022-0...",354,1
4,11,2022-03-24,"GetCreditInfo,UseLoanManage,GetCreditInfo,UseP...","2022-03-24 10:53:59,2022-03-24 10:54:07,2022-0...",692,1


In [41]:
# event, timestamp는 더이상 필요없으므로 삭제하기
new_log_event_df_merge.drop(['events', 'timestamp'], axis=1, inplace=True)

In [42]:
new_log_event_df_merge

,user_id,date,time,cluster
0,1,2022-05-03,7,1
1,1,2022-06-16,1,1
2,7,2022-05-22,0,1
3,9,2022-05-21,354,1
4,11,2022-03-24,692,1
...,...,...,...,...
3608000,879693,2022-06-29,0,5
3608001,879694,2022-03-31,655,2
3608002,879695,2022-05-27,171,2
3608003,879696,2022-03-14,400,0


In [43]:
new_log_event_df_merge.to_csv('data/new_log_event_cluster.csv', index=False)